In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
# fast_multi_category.py
# Ускоренный сбор: параллельный парсинг карточек с контролью RPS + бэкофф/ретраи
import os, time, math, random, threading, queue
from concurrent.futures import ThreadPoolExecutor, as_completed
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse

import requests
from bs4 import BeautifulSoup

from bg_parser import extract_product, save_csv  # твои функции

# ========= Настройки скорости =========
MAX_WORKERS = 8               # стартовое число потоков (будем адаптировать вниз при 429)
TARGET_RPS = 5.0              # целевая средняя нагрузка запросов/сек на карточки
BURST = 3.0                   # разрешённый «всплеск» токенов (размер бакета)
RETRY_MAX = 3                 # ретраи на карточку
BATCH_FLUSH = 100             # сколько удачных карточек копить перед флашем в CSV
CSV_PATH = "books_fast_test.csv"
APPEND = True                # первый флаш перезаписывает CSV

# ========= Сессионные заголовки (как браузер) =========
UA_POOL = [
    ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
     "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"),
    ("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
     "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"),
]
BASE_HEADERS = {
    "Accept-Language": "ru-RU,ru;q=0.9,en-US;q=0.8",
    "Referer": "https://www.biblio-globus.ru/",
    "Connection": "keep-alive",
}

# ========= Лимитер: токен-бакет =========
class TokenBucket:
    def __init__(self, rate: float, burst: float):
        self.rate = rate
        self.capacity = burst
        self.tokens = burst
        self.lock = threading.Lock()
        self.ts = time.perf_counter()

    def wait(self):
        # Блокируемся, пока не появится токен
        while True:
            with self.lock:
                now = time.perf_counter()
                elapsed = now - self.ts
                self.ts = now
                self.tokens = min(self.capacity, self.tokens + elapsed * self.rate)
                if self.tokens >= 1.0:
                    self.tokens -= 1.0
                    return
            # короткий сон, чтобы не крутить CPU
            time.sleep(0.005)

bucket = TokenBucket(TARGET_RPS, BURST)

# ========= Помощники каталога =========
def set_page_param(category_url: str, page_num: int) -> str:
    parts = urlparse(category_url)
    q = parse_qs(parts.query)
    q["page"] = [str(page_num)]
    return urlunparse(parts._replace(query=urlencode(q, doseq=True)))

def get_soup(url: str, session: requests.Session) -> BeautifulSoup:
    bucket.wait()  # лимитируем скорость на загрузку страниц каталога тоже
    r = session.get(url, timeout=20)
    r.raise_for_status()
    return BeautifulSoup(r.text, "lxml")

def extract_product_ids_from_catalog_soup(soup: BeautifulSoup) -> list[int]:
    ids = set()
    for a in soup.find_all("a", href=True):
        href = a["href"]
        if "/product/" in href:
            pid = href.split("/product/")[1].split("?")[0].strip("/")
            if pid.isdigit():
                ids.add(int(pid))
    if not ids:
        html = soup.decode()
        import re
        for pid in re.findall(r"/product/(\d+)", html):
            ids.add(int(pid))
    return sorted(ids)

def collect_product_urls_from_catalog(category_url: str,
                                      pages: int,
                                      session: requests.Session) -> list[str]:
    urls, seen, empty_streak = [], set(), 0
    for p in range(1, pages + 1):
        page_url = set_page_param(category_url, p)
        try:
            soup = get_soup(page_url, session)
        except Exception as e:
            print(f"[catalog p={p}] ❌ {e}")
            break
        page_ids = extract_product_ids_from_catalog_soup(soup)
        new_ids = [pid for pid in page_ids if pid not in seen]
        if not new_ids:
            empty_streak += 1
            print(f"[catalog p={p}] 0 новых (x{empty_streak})")
            if empty_streak >= 2:
                break
        else:
            empty_streak = 0
            for pid in new_ids:
                seen.add(pid)
                urls.append(f"https://www.biblio-globus.ru/product/{pid}")
            print(f"[catalog p={p}] +{len(new_ids)} (всего {len(urls)})")
        # мягкая пауза между страницами
        time.sleep(random.uniform(0.15, 0.35))
    return urls

# ========= Пул сессий (по одной на поток) =========
_tls = threading.local()
def get_session() -> requests.Session:
    if getattr(_tls, "sess", None) is None:
        s = requests.Session()
        s.headers.update(BASE_HEADERS | {"User-Agent": random.choice(UA_POOL)})
        _tls.sess = s
    return _tls.sess

# ========= Воркер по карточке с ретраями/бэкоффом =========
def fetch_card(idx: int, url: str):
    # лимитируем перед КАЖДЫМ заходом в extract_product (который сам делает запросы)
    # будем ретраить по 429/5xx/сетевым
    delay = 0.0
    for attempt in range(1, RETRY_MAX + 1):
        if delay:
            time.sleep(delay + random.uniform(0.0, 0.05))
        bucket.wait()
        try:
            row = extract_product(url)  # твоя функция, внутри делает requests.get
            if row is None:  # не книга/аудио
                # >>> изменено: добавлен вывод о пропуске
                print(f"{idx:04d}. ⏭️ пропущено (не книга/аудио) {url}")
                # <<< изменено
            return ("ok", row)
        except requests.HTTPError as e:
            code = e.response.status_code if e.response else None
            if code == 429 or (code and 500 <= code < 600):
                # экспоненциальный бэкофф
                delay = min(2.0, 0.4 * (2 ** (attempt - 1)))
                continue
            return ("err", f"HTTP {code}")
        except requests.RequestException as e:
            delay = min(1.5, 0.3 * (2 ** (attempt - 1)))
            continue
        except Exception as e:
            return ("err", str(e))
    return ("err", "max-retries")

# ========= Основной раннер =========
def run_fast(categories: list[str],
             pages_per_category: int = 10,
             total_limit: int | None = None):
    global APPEND

    # 1) Собираем product-URL’ы со всех категорий (последовательно, быстро)
    sess = get_session()
    all_urls = []
    for ci, cat in enumerate(categories, 1):
        print(f"\n== Категория {ci}/{len(categories)} ==\n{cat}")
        urls = collect_product_urls_from_catalog(cat, pages_per_category, sess)
        print(f"  → найдено ссылок: {len(urls)}")
        all_urls.extend(urls)
        # Хотите ограничивать уже тут? можно: if total_limit and len(all_urls) >= total_limit: break

    # 2) Параллельно парсим карточки
    kept = 0
    buf = []
    in_flight = 0
    workers = MAX_WORKERS

    def adapt_on_429(errmsg: str):
        nonlocal workers
        if "429" in errmsg and workers > 2:
            workers = max(2, workers - 1)
            print(f"⚠️ 429 получен — снижаем параллелизм до {workers}")

    # Гоняем пачками, чтобы можно было динамически менять число воркеров
    idx = 0
    while idx < len(all_urls):
        # берём окно URLs по текущему workers*2 (чуть «на вырост»)
        window = all_urls[idx: idx + workers * 2]
        idx += len(window)

        with ThreadPoolExecutor(max_workers=workers) as ex:
            futs = {ex.submit(fetch_card, i + 1, u): u for i, u in enumerate(window, start=idx - len(window) + 1)}
            for fut in as_completed(futs):
                url = futs[fut]
                try:
                    status, payload = fut.result()
                    if status == "ok":
                        row = payload
                        if row:
                            buf.append(row)
                            kept += 1
                            if len(buf) >= BATCH_FLUSH:
                                save_csv(buf, path=CSV_PATH, append=APPEND)
                                APPEND = True
                                buf.clear()
                                print(f"💾 промежуточно сохранено, всего книг: {kept}")
                        if total_limit and kept >= total_limit:
                            break
                    else:
                        print(f"❌ ошибка парсинга {url}: {payload}")
                        adapt_on_429(str(payload))  # <<< вернули снижение параллелизма
                except Exception as e:
                    print(f"❌ неожиданная ошибка {url}: {e}")
            if total_limit and kept >= total_limit:
                break

    # финальный флаш
    if buf:
        save_csv(buf, path=CSV_PATH, append=APPEND)

    print(f"\n🎉 Готово. Сохранено книг: {kept}. CSV → {CSV_PATH}")

# ===== Пример запуска =====
# if __name__ == "__main__":
#     CATEGORIES = [
#         "https://www.biblio-globus.ru/catalog/category?id=226&page=1&sort=0&instock=&isdiscount=",
#         # добавь ещё…
#     ]
#     run_fast(CATEGORIES, pages_per_category=8, total_limit=5000)


In [8]:
CATEGORIES = [
        "https://www.biblio-globus.ru/catalog/category?id=227&page=1&sort=0&instock=&isdiscount=",
        "https://www.biblio-globus.ru/catalog/category?id=241&page=1&sort=0&instock=&isdiscount="
        # добавь ещё…
    ]
run_fast(CATEGORIES, pages_per_category=150, total_limit=50000)


== Категория 1/2 ==
https://www.biblio-globus.ru/catalog/category?id=227&page=1&sort=0&instock=&isdiscount=
[catalog p=1] +12 (всего 12)
[catalog p=2] +12 (всего 24)
[catalog p=3] +12 (всего 36)
[catalog p=4] +12 (всего 48)
[catalog p=5] +12 (всего 60)
[catalog p=6] +12 (всего 72)
[catalog p=7] +12 (всего 84)
[catalog p=8] +12 (всего 96)
[catalog p=9] +12 (всего 108)
[catalog p=10] +12 (всего 120)
[catalog p=11] +12 (всего 132)
[catalog p=12] +12 (всего 144)
[catalog p=13] +12 (всего 156)
[catalog p=14] +12 (всего 168)
[catalog p=15] +12 (всего 180)
[catalog p=16] +12 (всего 192)
[catalog p=17] +12 (всего 204)
[catalog p=18] +12 (всего 216)
[catalog p=19] +12 (всего 228)
[catalog p=20] +12 (всего 240)
[catalog p=21] +12 (всего 252)
[catalog p=22] +12 (всего 264)
[catalog p=23] +12 (всего 276)
[catalog p=24] +12 (всего 288)
[catalog p=25] +12 (всего 300)
[catalog p=26] +12 (всего 312)
[catalog p=27] +12 (всего 324)
[catalog p=28] +12 (всего 336)
[catalog p=29] +12 (всего 348)
[catalog 

In [ ]:
CATEGORIES = [
        "https://www.biblio-globus.ru/catalog/category?id=226&page=1&sort=0&instock=&isdiscount=",
        "https://www.biblio-globus.ru/catalog/category?id=227&page=1&sort=0&instock=&isdiscount=",
        "https://www.biblio-globus.ru/catalog/category?id=241&page=1&sort=0&instock=&isdiscount="
        # добавь ещё…
    ]
run_fast(CATEGORIES, pages_per_category=5, total_limit=50000)


== Категория 1/3 ==
https://www.biblio-globus.ru/catalog/category?id=226&page=1&sort=0&instock=&isdiscount=
[catalog p=1] +12 (всего 12)
[catalog p=2] +12 (всего 24)
[catalog p=3] +12 (всего 36)
[catalog p=4] +12 (всего 48)
[catalog p=5] +12 (всего 60)
  → найдено ссылок: 60

== Категория 2/3 ==
https://www.biblio-globus.ru/catalog/category?id=227&page=1&sort=0&instock=&isdiscount=
[catalog p=1] +12 (всего 12)
[catalog p=2] +12 (всего 24)
[catalog p=3] +12 (всего 36)
[catalog p=4] +12 (всего 48)
[catalog p=5] +12 (всего 60)
  → найдено ссылок: 60

== Категория 3/3 ==
https://www.biblio-globus.ru/catalog/category?id=241&page=1&sort=0&instock=&isdiscount=
[catalog p=1] +12 (всего 12)
[catalog p=2] +12 (всего 24)
[catalog p=3] +12 (всего 36)
[catalog p=4] +12 (всего 48)
[catalog p=5] +12 (всего 60)
  → найдено ссылок: 60
0014. ⏭️ пропущено (не книга/аудио) https://www.biblio-globus.ru/product/11035081
0027. ⏭️ пропущено (не книга/аудио) https://www.biblio-globus.ru/product/11021965
0034.

In [4]:
CATEGORIES = [
        "https://www.biblio-globus.ru/catalog/category?id=202&page=1&sort=0&instock=&isdiscount=",
        # добавь ещё…
    ]
run_fast(CATEGORIES, pages_per_category=5, total_limit=50000)


== Категория 1/1 ==
https://www.biblio-globus.ru/catalog/category?id=202&page=1&sort=0&instock=&isdiscount=
[catalog p=1] +12 (всего 12)
[catalog p=2] +12 (всего 24)
[catalog p=3] +12 (всего 36)
[catalog p=4] +12 (всего 48)
[catalog p=5] +12 (всего 60)
  → найдено ссылок: 60
0012. ⏭️ пропущено (не книга/аудио) https://www.biblio-globus.ru/product/11053561
0035. ⏭️ пропущено (не книга/аудио) https://www.biblio-globus.ru/product/11023926
0044. ⏭️ пропущено (не книга/аудио) https://www.biblio-globus.ru/product/10903013
0036. ⏭️ пропущено (не книга/аудио) https://www.biblio-globus.ru/product/11027106

🎉 Готово. Сохранено книг: 56. CSV → books_fast_test.csv


In [4]:
import pandas as pd

# читаем твой CSV
df = pd.read_csv("subcategories.csv")

# допустим, колонка называется 'id'
category_ids = df["subcategories"].dropna().astype(int).tolist()

# шаблон ссылки
base_url = "https://www.biblio-globus.ru/catalog/category?id={}&page=1&sort=0&instock=&isdiscount="

# формируем список категорий
categories = [base_url.format(cid) for cid in category_ids]

print("Пример первых 5 ссылок:")
for c in categories[:20]:
    print(c)




Пример первых 5 ссылок:
https://www.biblio-globus.ru/catalog/category?id=182&page=1&sort=0&instock=&isdiscount=
https://www.biblio-globus.ru/catalog/category?id=185&page=1&sort=0&instock=&isdiscount=
https://www.biblio-globus.ru/catalog/category?id=186&page=1&sort=0&instock=&isdiscount=
https://www.biblio-globus.ru/catalog/category?id=189&page=1&sort=0&instock=&isdiscount=
https://www.biblio-globus.ru/catalog/category?id=6172&page=1&sort=0&instock=&isdiscount=
https://www.biblio-globus.ru/catalog/category?id=6175&page=1&sort=0&instock=&isdiscount=
https://www.biblio-globus.ru/catalog/category?id=6180&page=1&sort=0&instock=&isdiscount=
https://www.biblio-globus.ru/catalog/category?id=6181&page=1&sort=0&instock=&isdiscount=
https://www.biblio-globus.ru/catalog/category?id=6182&page=1&sort=0&instock=&isdiscount=
https://www.biblio-globus.ru/catalog/category?id=6183&page=1&sort=0&instock=&isdiscount=
https://www.biblio-globus.ru/catalog/category?id=6184&page=1&sort=0&instock=&isdiscount=
h

In [ ]:
run_fast(categories, pages_per_category=25, total_limit=70000)


== Категория 1/212 ==
https://www.biblio-globus.ru/catalog/category?id=182&page=1&sort=0&instock=&isdiscount=
[catalog p=1] +12 (всего 12)
[catalog p=2] +12 (всего 24)
[catalog p=3] +12 (всего 36)
[catalog p=4] +12 (всего 48)
[catalog p=5] +12 (всего 60)
[catalog p=6] +12 (всего 72)
[catalog p=7] +11 (всего 83)
[catalog p=8] +10 (всего 93)
[catalog p=9] +12 (всего 105)
[catalog p=10] +12 (всего 117)
[catalog p=11] +11 (всего 128)
[catalog p=12] +12 (всего 140)
[catalog p=13] +12 (всего 152)
[catalog p=14] +12 (всего 164)
[catalog p=15] +12 (всего 176)
[catalog p=16] +12 (всего 188)
[catalog p=17] +12 (всего 200)
[catalog p=18] +12 (всего 212)
[catalog p=19] +12 (всего 224)
[catalog p=20] +11 (всего 235)
[catalog p=21] +12 (всего 247)
[catalog p=22] +12 (всего 259)
[catalog p=23] +12 (всего 271)
[catalog p=24] +11 (всего 282)
[catalog p=25] +12 (всего 294)
  → найдено ссылок: 294

== Категория 2/212 ==
https://www.biblio-globus.ru/catalog/category?id=185&page=1&sort=0&instock=&isdiscou

In [6]:
categories_next = categories[26:36]
categories_next

['https://www.biblio-globus.ru/catalog/category?id=6152&page=1&sort=0&instock=&isdiscount=',
 'https://www.biblio-globus.ru/catalog/category?id=6156&page=1&sort=0&instock=&isdiscount=',
 'https://www.biblio-globus.ru/catalog/category?id=6163&page=1&sort=0&instock=&isdiscount=',
 'https://www.biblio-globus.ru/catalog/category?id=2422&page=1&sort=0&instock=&isdiscount=',
 'https://www.biblio-globus.ru/catalog/category?id=2463&page=1&sort=0&instock=&isdiscount=',
 'https://www.biblio-globus.ru/catalog/category?id=2472&page=1&sort=0&instock=&isdiscount=',
 'https://www.biblio-globus.ru/catalog/category?id=2478&page=1&sort=0&instock=&isdiscount=',
 'https://www.biblio-globus.ru/catalog/category?id=2490&page=1&sort=0&instock=&isdiscount=',
 'https://www.biblio-globus.ru/catalog/category?id=2496&page=1&sort=0&instock=&isdiscount=',
 'https://www.biblio-globus.ru/catalog/category?id=2507&page=1&sort=0&instock=&isdiscount=']

In [8]:
run_fast(categories_next, pages_per_category=25, total_limit=70000)


== Категория 1/10 ==
https://www.biblio-globus.ru/catalog/category?id=6152&page=1&sort=0&instock=&isdiscount=
[catalog p=1] +12 (всего 12)
[catalog p=2] +12 (всего 24)
[catalog p=3] +12 (всего 36)
[catalog p=4] +12 (всего 48)
[catalog p=5] +12 (всего 60)
[catalog p=6] +12 (всего 72)
[catalog p=7] +12 (всего 84)
[catalog p=8] +12 (всего 96)
[catalog p=9] +12 (всего 108)
[catalog p=10] +12 (всего 120)
[catalog p=11] +12 (всего 132)
[catalog p=12] +12 (всего 144)
[catalog p=13] +12 (всего 156)
[catalog p=14] +12 (всего 168)
[catalog p=15] +12 (всего 180)
[catalog p=16] +12 (всего 192)
[catalog p=17] +12 (всего 204)
[catalog p=18] +12 (всего 216)
[catalog p=19] +12 (всего 228)
[catalog p=20] +12 (всего 240)
[catalog p=21] +12 (всего 252)
[catalog p=22] +12 (всего 264)
[catalog p=23] +12 (всего 276)
[catalog p=24] +12 (всего 288)
[catalog p=25] +12 (всего 300)
  → найдено ссылок: 300

== Категория 2/10 ==
https://www.biblio-globus.ru/catalog/category?id=6156&page=1&sort=0&instock=&isdiscou

In [11]:
dfchek = pd.read_csv('books_fast_test.csv')
dfchek.shape

(9644, 5)

In [12]:
categories_next = categories[36:46]
run_fast(categories_next, pages_per_category=25, total_limit=70000)



== Категория 1/10 ==
https://www.biblio-globus.ru/catalog/category?id=6090&page=1&sort=0&instock=&isdiscount=
[catalog p=1] +12 (всего 12)
[catalog p=2] +12 (всего 24)
[catalog p=3] +12 (всего 36)
[catalog p=4] +12 (всего 48)
[catalog p=5] +12 (всего 60)
[catalog p=6] +12 (всего 72)
[catalog p=7] +12 (всего 84)
[catalog p=8] +12 (всего 96)
[catalog p=9] +12 (всего 108)
[catalog p=10] +12 (всего 120)
[catalog p=11] +12 (всего 132)
[catalog p=12] +12 (всего 144)
[catalog p=13] +12 (всего 156)
[catalog p=14] +12 (всего 168)
[catalog p=15] +12 (всего 180)
[catalog p=16] +12 (всего 192)
[catalog p=17] +12 (всего 204)
[catalog p=18] +12 (всего 216)
[catalog p=19] +12 (всего 228)
[catalog p=20] +12 (всего 240)
[catalog p=21] +12 (всего 252)
[catalog p=22] +12 (всего 264)
[catalog p=23] +12 (всего 276)
[catalog p=24] +12 (всего 288)
[catalog p=25] +12 (всего 300)
  → найдено ссылок: 300

== Категория 2/10 ==
https://www.biblio-globus.ru/catalog/category?id=6099&page=1&sort=0&instock=&isdiscou

In [13]:
categories_next = categories[56:66]
run_fast(categories_next, pages_per_category=25, total_limit=70000)


== Категория 1/10 ==
https://www.biblio-globus.ru/catalog/category?id=357&page=1&sort=0&instock=&isdiscount=
[catalog p=1] +12 (всего 12)
[catalog p=2] +12 (всего 24)
[catalog p=3] +12 (всего 36)
[catalog p=4] +12 (всего 48)
[catalog p=5] +12 (всего 60)
[catalog p=6] +12 (всего 72)
[catalog p=7] +12 (всего 84)
[catalog p=8] +9 (всего 93)
[catalog p=9] 0 новых (x1)
[catalog p=10] 0 новых (x2)
  → найдено ссылок: 93

== Категория 2/10 ==
https://www.biblio-globus.ru/catalog/category?id=359&page=1&sort=0&instock=&isdiscount=
[catalog p=1] +12 (всего 12)
[catalog p=2] +12 (всего 24)
[catalog p=3] +12 (всего 36)
[catalog p=4] +12 (всего 48)
[catalog p=5] +12 (всего 60)
[catalog p=6] +12 (всего 72)
[catalog p=7] +12 (всего 84)
[catalog p=8] +12 (всего 96)
[catalog p=9] +12 (всего 108)
[catalog p=10] +12 (всего 120)
[catalog p=11] +12 (всего 132)
[catalog p=12] +12 (всего 144)
[catalog p=13] +12 (всего 156)
[catalog p=14] +12 (всего 168)
[catalog p=15] +12 (всего 180)
[catalog p=16] +12 (все

In [14]:
categories_next = categories[66:76]
run_fast(categories_next, pages_per_category=25, total_limit=70000)


== Категория 1/10 ==
https://www.biblio-globus.ru/catalog/category?id=204&page=1&sort=0&instock=&isdiscount=
[catalog p=1] +12 (всего 12)
[catalog p=2] +12 (всего 24)
[catalog p=3] +12 (всего 36)
[catalog p=4] +12 (всего 48)
[catalog p=5] +12 (всего 60)
[catalog p=6] +12 (всего 72)
[catalog p=7] +12 (всего 84)
[catalog p=8] +12 (всего 96)
[catalog p=9] +12 (всего 108)
[catalog p=10] +12 (всего 120)
[catalog p=11] +12 (всего 132)
[catalog p=12] +12 (всего 144)
[catalog p=13] +12 (всего 156)
[catalog p=14] +12 (всего 168)
[catalog p=15] +12 (всего 180)
[catalog p=16] +12 (всего 192)
[catalog p=17] +12 (всего 204)
[catalog p=18] +12 (всего 216)
[catalog p=19] +12 (всего 228)
[catalog p=20] +12 (всего 240)
[catalog p=21] +12 (всего 252)
[catalog p=22] +12 (всего 264)
[catalog p=23] +12 (всего 276)
[catalog p=24] +12 (всего 288)
[catalog p=25] +12 (всего 300)
  → найдено ссылок: 300

== Категория 2/10 ==
https://www.biblio-globus.ru/catalog/category?id=205&page=1&sort=0&instock=&isdiscount

In [15]:
categories_next = categories[76:]
run_fast(categories_next, pages_per_category=25, total_limit=70000)


== Категория 1/136 ==
https://www.biblio-globus.ru/catalog/category?id=391&page=1&sort=0&instock=&isdiscount=
[catalog p=1] +12 (всего 12)
[catalog p=2] +12 (всего 24)
[catalog p=3] +12 (всего 36)
[catalog p=4] +12 (всего 48)
[catalog p=5] +12 (всего 60)
[catalog p=6] +12 (всего 72)
[catalog p=7] +12 (всего 84)
[catalog p=8] +12 (всего 96)
[catalog p=9] +12 (всего 108)
[catalog p=10] +12 (всего 120)
[catalog p=11] +12 (всего 132)
[catalog p=12] +12 (всего 144)
[catalog p=13] +12 (всего 156)
[catalog p=14] +12 (всего 168)
[catalog p=15] +12 (всего 180)
[catalog p=16] +12 (всего 192)
[catalog p=17] +12 (всего 204)
[catalog p=18] +12 (всего 216)
[catalog p=19] +12 (всего 228)
[catalog p=20] +12 (всего 240)
[catalog p=21] +12 (всего 252)
[catalog p=22] +12 (всего 264)
[catalog p=23] +12 (всего 276)
[catalog p=24] +12 (всего 288)
[catalog p=25] +12 (всего 300)
  → найдено ссылок: 300

== Категория 2/136 ==
https://www.biblio-globus.ru/catalog/category?id=392&page=1&sort=0&instock=&isdiscou

In [17]:
dfchek = pd.read_csv('books_fast_test.csv')
dfchek.shape

(39008, 5)

In [20]:
dfchek.sample(20)

,page_url,image_url,author,title,annotation
16161,https://www.biblio-globus.ru/product/10156015,https://static1.bgshop.ru/imagehandler.ashx?fi...,Лана Палей,Лучше чем йога. Гимнастика на каждый день,Предложенный в книге комплекс упражнений - это...
14227,https://www.biblio-globus.ru/product/11020347,https://static1.bgshop.ru/imagehandler.ashx?fi...,Александрова Ж.,Гид по искусству. История живописи,Гид по искусству - это инструмент для развития...
37906,https://www.biblio-globus.ru/product/10941452,https://static1.bgshop.ru/imagehandler.ashx?fi...,Пестерева М.Л.,Стандартные заключения и алгоритмы в практичес...,В учебном пособии представлены литературные и ...
7642,https://www.biblio-globus.ru/product/11050319,https://static1.bgshop.ru/imagehandler.ashx?fi...,Спиноза Б.,Этика,Бенедикт Спиноза — один из величайших философо...
3115,https://www.biblio-globus.ru/product/10299854,https://static1.bgshop.ru/imagehandler.ashx?fi...,Эрик Берн,"Люди, которые играют в игры","По мнению автора, жизнь каждого человека проте..."
31754,https://www.biblio-globus.ru/product/10607582,https://static1.bgshop.ru/imagehandler.ashx?fi...,Райан Норт,Как изобрести все. Создай цивилизацию с нуля,Настало время стать самым влиятельным человеко...
10353,https://www.biblio-globus.ru/product/11019570,https://static1.bgshop.ru/imagehandler.ashx?fi...,Юри У.,Мы можем договориться: Стратегии разрешения сл...,Переговорщик с 50-летним опытом и профессионал...
18045,https://www.biblio-globus.ru/product/11054682,https://static1.bgshop.ru/imagehandler.ashx?fi...,Кочелаева Л.Н.,Розы. Настенный календарь на 2026 год,"Календарь, наполненный нежностью и элегантност..."
38494,https://www.biblio-globus.ru/product/11020669,https://static1.bgshop.ru/imagehandler.ashx?fi...,NaN,Ля ты крыса! Офисная раскраска-антистресс,Эта раскраска поднимет вам настроение и поможе...
33853,https://www.biblio-globus.ru/product/10910369,https://static1.bgshop.ru/imagehandler.ashx?fi...,NaN,"А. А. Зайцев. Человек, инженер, ученый, министр…",Книга посвящена жизни и деятельности Анатолия ...


In [ ]:
keywords = ["календарь", "Календарь"]
pattern = "|".join(keywords)  # "календарь|Календарь"
unique_books_all = unique_books_all.drop(
    unique_books_all[unique_books_all["title"].str.contains(pattern, case=False, na=False)].index
)

In [22]:
df_test = dfchek

keywords = ["календарь", "Календарь"]
pattern = "|".join(keywords)  # "календарь|Календарь"
unique_books_all = df_test.drop(
    df_test[df_test["title"].str.contains(pattern, case=False, na=False)].index
)

In [24]:
unique_books_all.shape

(38831, 5)